<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Notebook for generating configuration for batch subscriptions in IBM Watson OpenScale in IBM Cloud Pak for Data v5.3

This notebook shows how to generate the following artefacts:
1. Configuration JSON needed to configure an IBM Watson OpenScale subscription.
2. Explainability Perturbations Archive
3. DDLs for creating Feedback, Payload and Explanations tables

The user needs to provide the necessary inputs (where marked) and download the generated artefacts. These artefacts 
have to be then uploaded to IBM Watson OpenScale UI. 

PS: This notebook can only generate artefacts for one model at a time. For multiple models, this notebook needs to be run for each model separately.

**Contents:**
1. [Installing Dependencies](#Installing-Dependencies)
2. [Select IBM Watson OpenScale Services](#Select-IBM-Watson-OpenScale-Services)
3. [Read sample scoring data](#Read-sample-scoring-data)
4. [Specify Model Inputs](#Specify-Model-Inputs)
5. [Generate Common Configuration](#Generate-Common-Configuration)
6. [Generate DDL for creating Scored Training data table](#Generate-DDL-for-creating-Scored-Training-data-table)
6. [Generate DDL for creating Feedback table](#Generate-DDL-for-creating-Feedback-table)
7. [Generate DDL for creating Payload table](#Generate-DDL-for-creating-Payload-table)
8. [Provide Spark Connection Details](#Provide-Spark-Connection-Details)
9. [Provide Storage Inputs](#Provide-Storage-Inputs)
10. [Provide Spark Resource Settings [Optional]](#Provide-Spark-Resource-Settings-[Optional])
11. [Provide Additional Spark Settings [Optional]](#Provide-Additional-Spark-Settings-[Optional])
12. [Provide Fairness Parameters [Optional]](#Provide-Fairness-Parameters-[Optional])
13. [Run Configuration Job](#Run-Configuration-Job)
14. [Download Configuration JSON](#Download-Configuration-JSON)
17. [Generate Perturbations csv](#Generate-Perturbations-csv)
18. [Generate DDL for creating Explanations Queue table](#Generate-DDL-for-creating-Explanations-Queue-table)
19. [Generate DDL for creating Explanations Table](#Generate-DDL-for-creating-Explanations-Table)
20. [Create Configuration Archive](#Create-Configuration-Archive)

### Installing Dependencies

In [ ]:
# Note: Restart kernel after the dependencies are installed
import sys

PYTHON = sys.executable

!pip install --no-warn-conflicts pyspark | tail -n 1  

**Note:** For IBM Watson OpenScale Cloud Pak for Data version 5.3

In [ ]:
# When this notebook is to be run on a zLinux cluster,
# install scikit-learn==1.6.1 using conda before installing ibm-wos-utils
# !conda install scikit-learn=1.6.1

!pip install --no-warn-conflicts "ibm-metrics-plugin~=5.3.0"

In [ ]:
# When this notebook is to be run on RT 24.1 environment, 
# uncomment the below line and execute.
# !pip install --upgrade "matplotlib~=3.10.1" transformers numexpr bottleneck opencv-python transformers numexpr bottleneck opencv-python --no-cache | tail -n 1

### Select IBM Watson OpenScale Services

Details of the service-specific flags available:

- ENABLE_QUALITY: Flag to allow generation of common configuration details needed if quality alone is selected
- ENABLE_FAIRNESS : Flag to allow generation of fairness specific data distribution needed for configuration
- ENABLE_EXPLAINABILITY : Flag to allow generation of explainability configuration and perturbations

In [ ]:
# ----------------------------------------------------------------------------------------------------
# IBM Confidential
# OCO Source Materials
# 5737-H76
# Copyright IBM Corp. 2021, 2025
# The source code for this Notebook is not published or other-wise divested of its trade
# secrets, irrespective of what has been deposited with the U.S.Copyright Office.
# ----------------------------------------------------------------------------------------------------

VERSION = "jdbc-1.1.12"

# Version history:

# jdbc-1.1.12 : Drift (classic) support is removed; Upgraded scikit-learn to 1.6.1; Upgraded ibm-wos-utils & ibm-metrics-plugin to 5.3.0.
# jdbc-1.1.11 : Global explainability support & Upgrade ibm-metrics-plugin to 5.2.1
# jdbc-1.1.10 : Postgress DB support & Upgrade ibm-wos-utils to 5.1.0
# jdbc-1.1.9 : Upgrade ibm-wos-utils to 5.0.0
# jdbc-1.1.8 : Upgrade ibm-wos-utils to 4.8.0
# jdbc-1.1.7 : Upgrade ibm-wos-utils to 4.7.0 (scikit-learn has been upgraded to 1.1.1)
# jdbc-1.1.6 : Upgrade ibm-wos-utils to 4.6.0 and update explainability archive with stats.
# jdbc-1.1.5 : Changed the way drift archive is created.
# jdbc-1.1.4 : Upgrade ibm-wos-utils to 4.1.1 (scikit-learn has been upgraded to 1.0.2)
# jdbc-1.1.3 : Add two drift tuning parameters: max_ranges_modifier and tail_discard_threshold; Upgrade ibm-wos-utils to 4.0.34
# jdbc-1.1.2 : Upgrade ibm-wos-utils to 4.0.31
# jdbc-1.1.1 : Add comment about conda install for zLinux environments; Upgrade ibm-wos-utils to 4.0.25
# jdbc-1.1   : Add partition information; Upgrade ibm-wos-utils to 4.0.24
# 1.0        : Initial release

In [ ]:
# Optional Input: Keep an identifiable name. This id is used to append to various table creation DDLs.
# A random UUID is used if this is not present.
# NOTEBOOK_RUN_ID = "some_identifiable_name"
NOTEBOOK_RUN_ID = None


# Service Configuration Flags
ENABLE_QUALITY = True
ENABLE_EXPLAINABILITY = True
ENABLE_FAIRNESS = True

RUN_JOB = ENABLE_QUALITY or ENABLE_EXPLAINABILITY or ENABLE_FAIRNESS

In [11]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName(
    "Common Configuration Generation").getOrCreate()

### Read sample scoring data

A sample scoring data is required to infer the schema of the complete data, so the size of the sample should be chosen accordingly. 

Additionally, the sample scoring data should have the following fields:
1. Feature Columns
2. Label/Target Column
3. Prediction Column (with same data type as the label column)
4. Probability Column (an array of model probabilities for all the class labels. Not required for regression models)


The sample data should be of type `pyspark.sql.dataframe.DataFrame`. The cell below gives samples on:
- how to read a CSV file from the local system into a Pyspark Dataframe.
- how to read parquet files in a directory from the local system into a Pyspark Dataframe.
- how to read orc files in a directory from the local system into a Pyspark Dataframe.

In [12]:
# Load a csv or a directory containing csv files as PySpark DataFrame
# spark_df = spark.read.csv("/path/to/dir/containing/csv/files", header=True, inferSchema=True)

# Load a directory containing parquet files as PySpark DataFrame
# spark_df = spark.read.parquet("/path/to/dir/containing/parquet/files")

# Load a directory containing orc files as PySpark DataFrame
# spark_df = spark.read.orc("/path/to/dir/containing/orc/files")

spark_df.printSchema()

root
 |-- CheckingStatus: string (nullable = true)
 |-- LoanDuration: integer (nullable = true)
 |-- CreditHistory: string (nullable = true)
 |-- LoanPurpose: string (nullable = true)
 |-- LoanAmount: integer (nullable = true)
 |-- ExistingSavings: string (nullable = true)
 |-- EmploymentDuration: string (nullable = true)
 |-- InstallmentPercent: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- OthersOnLoan: string (nullable = true)
 |-- CurrentResidenceDuration: integer (nullable = true)
 |-- OwnsProperty: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- InstallmentPlans: string (nullable = true)
 |-- Housing: string (nullable = true)
 |-- ExistingCreditsCount: integer (nullable = true)
 |-- Job: string (nullable = true)
 |-- Dependents: integer (nullable = true)
 |-- Telephone: string (nullable = true)
 |-- ForeignWorker: string (nullable = true)
 |-- Risk: string (nullable = true)
 |-- prediction: string (nullable = true)
 |-- probability: string (n

### Specify Model Inputs

#### Specify the Model Type

- Specify **binary** if the model is a binary classifier.
- Specify **multiclass** if the model is a multi-class classifier.
- Specify **regression** if the model is a regressor.

In [13]:
MODEL_TYPE = "binary"
# MODEL_TYPE = "multiclass"
# MODEL_TYPE = "regression"

#### Provide Column Details 

To proceed with this notebook, the following information is required.:

- **LABEL_COLUMN**: The column which contains the target field (also known as label column or the class label).
- **PREDICTION_COLUMN**: The column containing the model output. This should be of the same data type as the label column.
- **PROBABILITY_COLUMN**: The column (of type array) containing the model probabilities for all the possible prediction outcomes. This is not required for regression models.
- **CLASS_PROBABILITIES**: The columns (of type double) containing the model probabilities of class labels. This is not required for regression models. For example, for Go Sales model deployed in MS Azure ML Studio, value of this property would be `["Scored Probabilities for Class \"Camping Equipment\"", "Scored Probabilities for Class \"Mountaineering Equipment\"", "Scored Probabilities for Class \"Personal Accessories\""]`. Please note escaping double quotes is a must-have requirement for above example.
- **PARTITION_COLUMN**: The column to help Spark read and write data using multiple workers in your JDBC storage. This will help improve the performance of your Spark jobs. The default value is set to `wos_partition_column`. The cells below will include this column for generating CREATE TABLE DDLs and ALTER TABLE DDLs for your data source. This column will not be used for computation purposes.
- **PROTECTED_ATTRIBUTES**: [Optional] The columns which exist in training data but are not used to train the model. This is required to monitor fairness on non-feature columns i.e Indirect Bias.

- **JDBC_CONNECTION_TYPE**: JDBC Connection type used (supported types : db2, postgresql).

Note: Please be careful when choosing an existing feature column as partition column. If data in this feature column is not properly divided across various possible values, it could lead to data-skew problem with Spark computation. Which means, majority of data is sent to one worker for computation - leading to wastage of compute resources and increased computation time. It is recommended to use a column with monotonically increasing value as partition column.

In [14]:
LABEL_COLUMN = "<label_column>"
PREDICTION_COLUMN = "<model prediction column>"
PROBABILITY_COLUMN = "<model probability column. ignored in case of regression models>"
CLASS_PROBABILITIES = ["<list of columns containing class probabilities. Ignored in case of regression models>"]

PARTITION_COLUMN = "wos_partition_column"
# [Optional] Provide list of protected attributes i.e non-feature columns present in the data.
PROTECTED_ATTRIBUTES = []

Based on the sample data and key columns provided above, the notebook will deduce the feature columns and the categorical columns. They will be printed in the output of this cell. If you wish to make changes to them, you can do so in the subsequent cell.

In [15]:
from pyspark.sql.types import BooleanType, StringType

feature_columns = spark_df.columns.copy()
print(feature_columns)
feature_columns.remove(LABEL_COLUMN)
feature_columns.remove(PREDICTION_COLUMN)


if MODEL_TYPE != "regression":
    feature_columns.remove(PROBABILITY_COLUMN)

if PROTECTED_ATTRIBUTES:
    for protected_attribute in PROTECTED_ATTRIBUTES:
        feature_columns.remove(protected_attribute)

print("Feature Columns : {}".format(feature_columns))

categorical_columns = [f.name for f in spark_df.schema.fields if isinstance(f.dataType, (BooleanType, StringType)) and f.name in feature_columns]
print("Categorical Columns : {}".format(categorical_columns))

['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker', 'Risk', 'prediction', 'probability']
Feature Columns : ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker']
Categorical Columns : ['CheckingStatus', 'CreditHistory', 'LoanPurpose', 'ExistingSavings', 'EmploymentDuration', 'Sex', 'OthersOnLoan', 'OwnsProperty', 'InstallmentPlans', 'Housing', 'Job', 'Telephone', 'ForeignWorker']


In [16]:
config_info = {
    "problem_type": MODEL_TYPE,
    "model_type": MODEL_TYPE,
    "label_column": LABEL_COLUMN,
    "prediction": PREDICTION_COLUMN,
    "probability": PROBABILITY_COLUMN,
    "class_probabilities": CLASS_PROBABILITIES
}

config_info["feature_columns"] = feature_columns
config_info["categorical_columns"] = categorical_columns
config_info["protected_attributes"] = PROTECTED_ATTRIBUTES

In [17]:
from ibm_wos_utils.joblib.utils.notebook_utils import validate_config_info
validate_config_info(config_info)

### Generate Common Configuration

IBM Watson OpenScale requires two additional fields - a unique identifier for each record in your feedback/payload tables ("scoring_id") and a timestamp field ("scoring_timestamp") denoting when that record entered the table. These fields are automatically added in the common configuration. 

Please make sure that these fields are present in the respective tables.

In [18]:
from ibm_wos_utils.joblib.utils.notebook_utils import generate_schemas

common_config = config_info.copy()
common_configuration = generate_schemas(spark_df, common_config)

config_json = {}
config_json["common_configuration"] = common_configuration
config_json["batch_notebook_version"] = VERSION

In [19]:
from ibm_wos_utils.joblib.utils.notebook_utils import get_max_length_categories

max_length_categories = get_max_length_categories(spark_df)

### Choose the JDBC Connection type (supported: db2, postgresql)

In [ ]:
import importlib

def get_table_ddl_module(db_type):
    module = None
    if db_type == 'db2':
        module = importlib.import_module('ibm_wos_utils.joblib.utils.ddl_utils_db2')
    elif db_type == 'postgresql':
        module = importlib.import_module('ibm_wos_utils.joblib.utils.ddl_utils_postgres')

    return module

In [ ]:
JDBC_CONNECTION_TYPE = "<to_be_edited>" #Type of the JDBC Connection, supported types: db2, postgresql

table_ddl_module = get_table_ddl_module(JDBC_CONNECTION_TYPE)

### Generate DDL for creating Scored Training data table

In [20]:
# Schema Name where Scored Training Table should be created.
SCORED_TRAINING_SCHEMA_NAME = None

table_ddl_module.generate_scored_training_table_ddl(config_json, schema_name=SCORED_TRAINING_SCHEMA_NAME,\
                                 table_suffix=NOTEBOOK_RUN_ID, max_length_categories=max_length_categories, partition_column=PARTITION_COLUMN)

**Create Table SQL Query:** Run the following query, to create a new table
CREATE TABLE "notebooks_five0"."scored_training_table" ("CheckingStatus" VARCHAR(64), "LoanDuration" BIGINT, "CreditHistory" VARCHAR(64), "LoanPurpose" VARCHAR(64), "LoanAmount" BIGINT, "ExistingSavings" VARCHAR(64), "EmploymentDuration" VARCHAR(64), "InstallmentPercent" BIGINT, "Sex" VARCHAR(64), "OthersOnLoan" VARCHAR(64), "CurrentResidenceDuration" BIGINT, "OwnsProperty" VARCHAR(64), "Age" BIGINT, "InstallmentPlans" VARCHAR(64), "Housing" VARCHAR(64), "ExistingCreditsCount" BIGINT, "Job" VARCHAR(64), "Dependents" BIGINT, "Telephone" VARCHAR(64), "ForeignWorker" VARCHAR(64), "Risk" VARCHAR(64) NOT NULL, "prediction" VARCHAR(64) NOT NULL, "probability" VARCHAR(32000) NOT NULL, "" BIGINT GENERATED ALWAYS AS IDENTITY);

**Alter Table SQL Queries:** Run the following queries, in the given order, to add a partition column in your existing table.
1. `ALTER TABLE "notebooks_five0"."scored_training_table" ADD COLUMN "" BIGINT NOT NULL DEFAULT 0;`
2. `ALTER TABLE "notebooks_five0"."scored_training_table" ALTER COLUMN "" DROP DEFAULT;`
3. `ALTER TABLE "notebooks_five0"."scored_training_table" ALTER COLUMN "" SET GENERATED ALWAYS AS IDENTITY;`
4. `CALL SYSPROC.ADMIN_CMD('REORG TABLE "notebooks_five0"."scored_training_table"');`
5. `UPDATE "notebooks_five0"."scored_training_table" SET "" = DEFAULT;`

### Generate DDL for creating Feedback table


In [21]:
# Schema Name where Feedback Table should be created.
FEEDBACK_SCHEMA_NAME = None

if ENABLE_QUALITY:
    table_ddl_module.generate_feedback_table_ddl(config_json, schema_name=FEEDBACK_SCHEMA_NAME,\
                                 table_suffix=NOTEBOOK_RUN_ID, max_length_categories=max_length_categories, partition_column=PARTITION_COLUMN)

**Create Table SQL Query:** Run the following query, to create a new table
- `CREATE TABLE "feedback_five0"."feedback_table" ("CheckingStatus" VARCHAR(64), "LoanDuration" BIGINT, "CreditHistory" VARCHAR(64), "LoanPurpose" VARCHAR(64), "LoanAmount" BIGINT, "ExistingSavings" VARCHAR(64), "EmploymentDuration" VARCHAR(64), "InstallmentPercent" BIGINT, "Sex" VARCHAR(64), "OthersOnLoan" VARCHAR(64), "CurrentResidenceDuration" BIGINT, "OwnsProperty" VARCHAR(64), "Age" BIGINT, "InstallmentPlans" VARCHAR(64), "Housing" VARCHAR(64), "ExistingCreditsCount" BIGINT, "Job" VARCHAR(64), "Dependents" BIGINT, "Telephone" VARCHAR(64), "ForeignWorker" VARCHAR(64), "Risk" VARCHAR(64) NOT NULL, "prediction" VARCHAR(64) NOT NULL, "probability" VARCHAR(32000) NOT NULL, "scoring_id" VARCHAR(64) NOT NULL PRIMARY KEY, "scoring_timestamp" TIMESTAMP NOT NULL, "" BIGINT GENERATED ALWAYS AS IDENTITY);`

- `CREATE INDEX "feedback_table_index" ON "feedback_five0"."feedback_table" ("scoring_timestamp" DESC)`

**Alter Table SQL Queries:** Run the following queries, in the given order, to add a partition column in your existing table.
1. `ALTER TABLE "feedback_five0"."feedback_table" ADD COLUMN "" BIGINT NOT NULL DEFAULT 0;`
2. `ALTER TABLE "feedback_five0"."feedback_table" ALTER COLUMN "" DROP DEFAULT;`
3. `ALTER TABLE "feedback_five0"."feedback_table" ALTER COLUMN "" SET GENERATED ALWAYS AS IDENTITY;`
4. `CALL SYSPROC.ADMIN_CMD('REORG TABLE "feedback_five0"."feedback_table"');`
5. `UPDATE "feedback_five0"."feedback_table" SET "" = DEFAULT;`

### Generate DDL for creating Payload table


In [ ]:
# Schema Name where Payload Table should be created.
PAYLOAD_SCHEMA_NAME = None

if ENABLE_EXPLAINABILITY or ENABLE_FAIRNESS:
    table_ddl_module.generate_payload_table_ddl(config_json, schema_name=PAYLOAD_SCHEMA_NAME,\
                                table_suffix=NOTEBOOK_RUN_ID, max_length_categories=max_length_categories, partition_column=PARTITION_COLUMN)

**Create Table SQL Query:** Run the following query, to create a new table
- `CREATE TABLE "payload_five0"."payload_table" ("CheckingStatus" VARCHAR(64), "LoanDuration" BIGINT, "CreditHistory" VARCHAR(64), "LoanPurpose" VARCHAR(64), "LoanAmount" BIGINT, "ExistingSavings" VARCHAR(64), "EmploymentDuration" VARCHAR(64), "InstallmentPercent" BIGINT, "Sex" VARCHAR(64), "OthersOnLoan" VARCHAR(64), "CurrentResidenceDuration" BIGINT, "OwnsProperty" VARCHAR(64), "Age" BIGINT, "InstallmentPlans" VARCHAR(64), "Housing" VARCHAR(64), "ExistingCreditsCount" BIGINT, "Job" VARCHAR(64), "Dependents" BIGINT, "Telephone" VARCHAR(64), "ForeignWorker" VARCHAR(64), "prediction" VARCHAR(64) NOT NULL, "probability" VARCHAR(32000) NOT NULL, "scoring_id" VARCHAR(64) NOT NULL PRIMARY KEY, "scoring_timestamp" TIMESTAMP NOT NULL, "" BIGINT GENERATED ALWAYS AS IDENTITY);`

- `CREATE INDEX "payload_table_index" ON "payload_five0"."payload_table" ("scoring_timestamp" DESC)`

**Alter Table SQL Queries:** Run the following queries, in the given order, to add a partition column in your existing table.
1. `ALTER TABLE "payload_five0"."payload_table" ADD COLUMN "" BIGINT NOT NULL DEFAULT 0;`
2. `ALTER TABLE "payload_five0"."payload_table" ALTER COLUMN "" DROP DEFAULT;`
3. `ALTER TABLE "payload_five0"."payload_table" ALTER COLUMN "" SET GENERATED ALWAYS AS IDENTITY;`
4. `CALL SYSPROC.ADMIN_CMD('REORG TABLE "payload_five0"."payload_table"');`
5. `UPDATE "payload_five0"."payload_table" SET "" = DEFAULT;`

### Provide Spark Connection Details

1. If your job is going to run on Spark cluster as part of an IBM Analytics Engine instance on IBM Cloud Pak for Data, enter the following details:
    
    - **IAE_SPARK_DISPLAY_NAME**: Display Name of the Spark instance in IBM Analytics Engine
    - **IAE_SPARK_JOBS_ENDPOINT**: Spark Jobs Endpoint for IBM Analytics Engine
    - **IBM_CPD_VOLUME**: IBM Cloud Pak for Data storage volume name
    - **IBM_CPD_USERNAME**: IBM Cloud Pak for Data username
    - **IBM_CPD_APIKEY**: IBM Cloud Pak for Data API key


2. If your job is going to run on Spark Cluster as part of a Remote Hadoop Ecosystem, enter the following details:

    - **SPARK_MANAGER_ENDPOINT**: Endpoint URL where the Spark Manager Application is running
    - **SPARK_MANAGER_USERNAME**: Username to connect to Spark Manager Application
    - **SPARK_MANAGER_PASSWORD**: Password to connect to Spark Manager Application

#### Credentials Block for Spark in IAE

In [23]:
from ibm_wos_utils.joblib.utils.constants import SparkType

IAE_SPARK_DISPLAY_NAME = "<Display Name of the Spark instance in IBM Analytics Engine>"
IAE_SPARK_JOBS_ENDPOINT = "<Spark Jobs Endpoint for IBM Analytics Engine>"
IBM_CPD_VOLUME = "<IBM Cloud Pak for Data storage volume name>"
IBM_CPD_USERNAME = "<IBM Cloud Pak for Data username>"
IBM_CPD_APIKEY = "<IBM Cloud Pak for Data API key>"

# Credentials Block for Spark in IAE
credentials = {
    "connection": {
        "display_name": IAE_SPARK_DISPLAY_NAME,
        "endpoint": IAE_SPARK_JOBS_ENDPOINT,
        "location_type": SparkType.IAE_SPARK.value,
        "volume": IBM_CPD_VOLUME
    },
    "credentials": {
        "username": IBM_CPD_USERNAME,
        "apikey": IBM_CPD_APIKEY
    }
}

#### Credentials Block for Spark in Remote Hadoop Ecosystem

In [24]:
from ibm_wos_utils.joblib.utils.constants import SparkType

SPARK_MANAGER_ENDPOINT = "<Endpoint URL where Spark Manager Application is running>"
SPARK_MANAGER_USERNAME = "<Username to connect to Spark Manager Application>"
SPARK_MANAGER_PASSWORD = "<Password to connect to Spark Manager Application>"

# Credentials Block for Spark in Remote Hadoop Ecosystem
credentials = {
    "connection": {
        "endpoint": SPARK_MANAGER_ENDPOINT,
        "location_type": SparkType.REMOTE_SPARK.value
    },
    "credentials": {
        "username": SPARK_MANAGER_USERNAME,
        "password": SPARK_MANAGER_PASSWORD
    }
}

### Provide Spark Resource Settings. [Optional]

Configure how much of your Spark Cluster resources can this job consume.

In [25]:
"""
spark_settings = {
    # max_num_executors: Maximum Number of executors to launch for this session
    "max_num_executors": "2",
    
    # min_executors: Minimum Number of executors to launch for this session
    "min_executors": "1",
    
    # executor_cores: Number of cores to use for each executor
    "executor_cores": "2",
    
    # executor_memory: Amount of memory (in GBs) to use per executor process
    "executor_memory": "2",
    
    # driver_cores: Number of cores to use for the driver process
    "driver_cores": "2",
    
    # driver_memory: Amount of memory (in GBs) to use for the driver process 
    "driver_memory": "1"
}
"""

spark_settings = None

### Provide Additional Spark Settings [Optional]

Any other Spark property that can be set via **SparkConf**, provide them in the next cell. These properties are sent to the Spark cluster verbatim. Leave the variable `conf` to `None` or `{}` if no additional property is required.

- [A list of available properties for Spark 2.4.6](https://spark.apache.org/docs/2.4.6/configuration.html#available-properties)

In [26]:
"""
conf = {
    "spark.yarn.maxAppAttempts": 1
}

"""

conf = None

### Provide Storage Inputs

Enter DB2 Storage details.
 - **JDBC_HOST**: Hostname of the JDBC Connection
 - **JDBC_PORT**: Port of the JDBC Connection
 - **JDBC_USE_SSL**: Boolean Flag to indicate whether to use SSL while connecting.
 - **JDBC_SSL_CERTIFICATE**: SSL Certificate [Base64 encoded string] of the JDBC Connection. Ignored if JDBC_USE_SSL is False.
 - **JDBC_DRIVER**: Optional. Class name of the JDBC driver to use to connect e.g. for DB2 use com.ibm.db2.jcc.DB2Driver
 - **JDBC_USERNAME**: Username of the JDBC Connection
 - **JDBC_PASSWORD**: Password of the JDBC Connection
 - **JDBC_DATABASE_NAME**: Name of the JDBC Database to use to connect.
 - **TRAINING_SCHEMA_NAME**: Name of the JDBC Schema that has training table.
 - **TRAINING_TABLE_NAME**: Name of the JDBC Table that has the scored training data.




In [27]:
JDBC_HOST = "<Hostname of the JDBC Connection>"
JDBC_PORT = "<Port of the JDBC Connection>"
JDBC_USE_SSL = "<Boolean Flag to indicate whether to use SSL while connecting.>"
JDBC_SSL_CERTIFICATE = "<SSL Certificate [Base64 encoded string] of the JDBC Connection. Ignored if JDBC_USE_SSL is False.>"
JDBC_DRIVER = "<Optional. Class name of the JDBC driver to use to connect e.g. for DB2 use com.ibm.db2.jcc.DB2Driver>"
JDBC_USERNAME = "<Username of the JDBC Connection>"
JDBC_PASSWORD = "<Password of the JDBC Connection>"

JDBC_DATABASE_NAME = "<Name of the JDBC Database to use to connect.>"
TRAINING_SCHEMA_NAME = "<Name of the JDBC Schema that has training table.>"
TRAINING_TABLE_NAME = "<Name of the JDBC Table that has the scored training data.>"

In [28]:
num_partitions_recommended = 12

if spark_settings:
    executors = int(spark_settings.get("max_num_executors", 2))
    cores = int(spark_settings.get("executor_cores", 2))
    num_partitions_recommended = 3 * executors * cores
    
print("{} is the recommended value for number of partitions in your data. Please change this value as per your data.".format(num_partitions_recommended))

12 is the recommended value for number of partitions in your data. Please change this value as per your data.


- **NUM_PARTITIONS**: The maximum number of partitions that Spark can divide the data into. In JDBC, it also means the maximum number of connections that Spark can make to the JDBC store for reading/writing data. 

The recommended value is calculated in the above cell as a multiple of total workers allotted for this job. You can use the same value or change it in the next cell.

In [29]:
NUM_PARTITIONS = num_partitions_recommended

In [30]:
jdbc_url = "jdbc:{}://{}:{}/{}".format(JDBC_CONNECTION_TYPE, JDBC_HOST, JDBC_PORT, JDBC_DATABASE_NAME)

storage_details = {
    "type": "jdbc",
    "connection": {
        "jdbc_driver": JDBC_DRIVER,
        "jdbc_url": jdbc_url,
        "use_ssl": JDBC_USE_SSL,
        "certificate": JDBC_SSL_CERTIFICATE,
        "location_type": "jdbc"
    },
    "credentials":{
        "username": JDBC_USERNAME,
        "password": JDBC_PASSWORD,
    }
}

tables = [
    {
        "database": JDBC_DATABASE_NAME,
        "schema": TRAINING_SCHEMA_NAME,
        "table": TRAINING_TABLE_NAME,
        "type": "training",
        "parameters": {
            "partition_column": PARTITION_COLUMN,
            "num_partitions": NUM_PARTITIONS
        }
    }
]

### Provide Fairness Parameters [REQUIRED if `ENABLE_FAIRNESS` is set to True]

Provide the fairness parameters in this cell. Leave the variable `fairness_parameters` to `None` or `{}` if fairness is not to be enabled.

In [32]:
"""
fairness_parameters = {
    "features": [
        {
            "feature": "<The fairness attribute name>", # The feature on which the fairness check is to be done
            "majority": [<majority groups/ranges for categorical/numerical columns respectively>],
            "minority": [<minority groups/ranges for categorical/numerical columns respectively>],
            "threshold": <The threshold value between 0 and 1> [OPTIONAL, default value is 0.8]
        }
    ],
    "class_label": LABEL_COLUMN,
    "favourable_class": [<favourable classes/ranges for classification/regression models repectively>],
    "unfavourable_class": [<unfavourable classes/ranges for classification/regression models repectively>],
    "min_records": <The minimum number of records on which the fairness check is to be done> [OPTIONAL]

    # The following parameters are only supported for subscriptions with a synchronous scoring endpoint.
    
    "perform_perturbation": <(Boolean) Whether the user wants to calculate the balanced (payload + perturbed) data.>,
    "sample_size_percent": <(Integer 1-100) How much percentage of data to be read for balanced data calculation.>,
    "numerical_perturb_count_per_row": <[Optional] The number of perturbed rows to be generated per row for numerical perturbation. [Default: 2]>,
    "float_decimal_place_precision": <[Optional] The decimal place precision to be used for numerical perturbation when data is float.>,
    "numerical_perturb_seed": <[Optional] The seed to be used for numerical perturbation while picking up random values.>,
    "scoring_page_size": <[Optional] The size of the page in the number of rows. [Default: 1000]>
}
"""

fairness_parameters = None

#### Provide Explainability Parameters [REQUIRED if `ENABLE_EXPLAINABILITY` is set to True]

Provide the explainability parameters in this cell. Leave the variable `explainability_parameters` to `None` or `{}` if explainability is not to be enabled.

In [ ]:
"""
# Set the below explainability parameters to enable lime global explanation generation.
# Note: When LIME global explanation is enabled, the explainability archive upload and explainability monitor enablement should be done using python sdk/api. 
# LIME global explanation configuration is not supported from IBM Watson OpenScale GUI.
explainability_parameters = {
    "lime":{ # specify this attribute only if you want to generate lime global or local explanations
            "perturbations_count": 10000 # default value for the number of perturbations to be generated.
        },
    "global_explanation": {
        "enabled": True, # Enable global explanation
        "explanation_method": "lime", # The explanation method to use
        "training_data_sample_size": 1000, # [Optional] The sample size of records to be used for generating training data global explanation. If not specified entire training data is used.
        "sample_size": 1000, # [Optional] The sample size of records to be used for generating payload data global explanation. If not specified entire data in the payload window is used.
    }
}
"""

explainability_parameters = None

### Run Configuration Job

In [ ]:
SHOW_PROGRESS = True

arguments = {
    "batch_notebook_version": VERSION,
    "common_configuration" : common_configuration,
    "enable_explainability": ENABLE_EXPLAINABILITY,
    "enable_fairness": ENABLE_FAIRNESS,
    "monitoring_run_id": NOTEBOOK_RUN_ID,
    "storage": storage_details,
    "tables": tables,
    "show_progress": SHOW_PROGRESS
}
if ENABLE_FAIRNESS:
    if fairness_parameters is None or fairness_parameters == {}:
        raise ValueError("Fairness parameters are required if fairness is enabled.")
    arguments["fairness_parameters"] = fairness_parameters

if ENABLE_EXPLAINABILITY:
    if explainability_parameters is None or explainability_parameters == {}:
        raise ValueError("Explainability parameters are required if explainability is enabled.")
    arguments["explainability_parameters"] = explainability_parameters
    
job_params = {
    "arguments": arguments,
    "spark_settings": spark_settings,
    "dependency_zip": [],
    "conf": conf
}

The following cell will run the Configuration job. If `SHOW_PROGRESS` is `True`, it will also print the status of job in the output section. Please wait for the status to be **FINISHED**.

A successful job status goes through the following values:
1. STARTED
2. Explainability Configuration STARTED
3. Explainability Configuration COMPLETED
4. Fairness Configuration STARTED
5. Fairness Configuration COMPLETED
6. FINISHED

If at anytime there is a failure, you will see a **FAILED** status with an exception trace. 

In [34]:
from ibm_wos_utils.joblib.clients.engine_client import EngineClient
from ibm_wos_utils.common.batch.jobs.configuration import Configuration
from ibm_wos_utils.joblib.utils.notebook_utils import JobStatus


if RUN_JOB:
    job_name="Configuration_Job"
    client = EngineClient(credentials=credentials)
    job_response = client.engine.run_job(job_name=job_name, job_class=Configuration,
                                        job_args=job_params, background=True)
    # Print Job Status.
    if SHOW_PROGRESS:
        JobStatus(client, job_response).print_status()

Application ID: None; Job ID: 40d3fffb-02e7-4b5f-8f45-70d5d47fbd43; Status: FINISHED.
Total Run Time: 4 minutes 34 seconds 


If `SHOW_PROGRESS` is `False`, you can run the below cell to check the job status at any point manually.

In [35]:
if not SHOW_PROGRESS and RUN_JOB:
    job_id = job_response.get("id")
    print(client.engine.get_job_status(job_id))

### Download Configuration JSON

In [36]:
import json
from ibm_wos_utils.joblib.utils.notebook_utils import create_download_link

if RUN_JOB:
        configuration = client.engine.get_file(job_response.get(
                "output_file_path") + "/configuration.json")
        config=json.loads(json.loads(configuration).get("configuration"))
else:
        config = config_json

# handle class probabilities explicitly
from ibm_wos_utils.joblib.utils.param_utils import get

class_probabilities = get(common_configuration, "class_probabilities")
if class_probabilities:
    # clean up any class probability columns already added
    updated_output_data_schema_fields = []
    for field in get(config, "common_configuration.output_data_schema.fields"):
        if get(field, "metadata.modeling_role") == "class_probability":
            continue

        updated_output_data_schema_fields.append(field)

    # add class probabilities to output_data_schema
    for class_probability in class_probabilities:
        updated_output_data_schema_fields.append({
            "name": class_probability,
            "type": "double",
            "nullable": True,
            "metadata": {
                "modeling_role": "class_probability"
            }
        })

    config["common_configuration"]["output_data_schema"]["fields"] = updated_output_data_schema_fields
    config["common_configuration"]["probability_fields"] = class_probabilities

display(create_download_link(config, "config"))

### Generate Perturbations csv

In [42]:
import pandas as pd
from ibm_wos_utils.explainability.utils.perturbations import Perturbations

if ENABLE_EXPLAINABILITY:
    perturbations_count = 10000 # Default value for the number of perturbations to be generated as part of perturbations.csv file.
    # Please modify the perturbations_count variable according to your usecase.
    perturbations=Perturbations(training_stats=config.get("explainability_configuration"), problem_type=MODEL_TYPE, perturbations_count=perturbations_count)
    perturbs_df = perturbations.generate_perturbations()
    perturbs_df.to_csv("perturbations.csv",index=False)

The perturbations required for explainability are stored in the file perturbations.csv in the above step.
The user should score these perturbations against the user model and provide the scoring output as a dataframe with **probability** and **prediction** columns.

Please note that the probability and prediction column names in the data frame should be same as PREDICTION_COLUMN and PROBABILITY_COLUMN provided in this notebook.

Note: For regression model probability column is not required.

In [ ]:
%%time
def run_training_data_global_expln_job():
    from ibm_metrics_plugin.common.utils.configuration_utility import ConfigurationUtility
    config_json["common_configuration"]["explainability_parameters"] = explainability_parameters
    common_params = config_json["common_configuration"]
    training_data_connection = {}
    training_data_connection["storage_details"] = storage_details
    training_data_connection["tables"] = tables
    
    credentials["spark_settings"] = spark_settings or None
    
    config_utility = ConfigurationUtility(common_params, training_data_connection, credentials)
    global_explanation = config_utility.get_training_global_explanation(engine_client=client,
                                                                              training_stats=config.get("explainability_configuration"),
                                                                              lime_scored_perturbations=lime_scored_perturbations,
                                                                              common_config=common_configuration)
    return global_explanation

In [ ]:
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()
wslib.download_file("scored_perturbations.csv")

In [ ]:
from ibm_wos_utils.joblib.utils.notebook_utils import create_archive
from json import dumps
import pandas as pd
import numpy as np
if ENABLE_EXPLAINABILITY:
    def convert(value):
        # Avoid converting lists to a string
        try:
            parsed_value = json.loads(value)  # Convert stringified lists back to Python lists
            return parsed_value if isinstance(parsed_value, list) else value
        except (json.JSONDecodeError, TypeError):
            return value
    # Read the scored_perturbations csv
    df = pd.read_csv("scored_perturbations.csv")
    if config_info.get("problem_type") == "regression":
        lime_scored_perturbations = {
            "predictions": np.array(df[PREDICTION_COLUMN]).tolist()
        }
    else:
        lime_scored_perturbations = {
            "probabilities": [convert(val) for val in df[PROBABILITY_COLUMN]],
            "predictions": np.array(df[PREDICTION_COLUMN]).tolist()
            }

    archive_data = {
        "configuration.json": dumps({"parameters": explainability_parameters}),
        "lime_scored_perturbations.json": dumps(lime_scored_perturbations),
        "training_statistics.json": dumps({"training_statistics": config.get("explainability_configuration")})
    }

    if explainability_parameters["global_explanation"]["enabled"]:
        training_global_explanation = run_training_data_global_expln_job()
        archive_data["lime_training_data_global_explanation.json"] = training_global_explanation
    
    display(create_archive(data=archive_data, archive_name="explainability"))

### Generate DDL for creating Explanations Queue table [Optional]

Provide details for creating a separate Explanations Queue table. IBM Watson OpenScale will be generating Explanations for all the transactions in this table. Alternatively, the payload table created in the notebook above can also be used for this purpose.

In [ ]:
# Database Name where Explanations Queue Table should be created.
EXPLANATIONS_QUEUE_SCHEMA_NAME = None

if ENABLE_EXPLAINABILITY:
    table_ddl_module.generate_payload_table_ddl(config_json, schema_name=EXPLANATIONS_QUEUE_SCHEMA_NAME,\
                                            table_prefix="explanations_queue", table_suffix=NOTEBOOK_RUN_ID,
                                            max_length_categories=max_length_categories, partition_column=PARTITION_COLUMN)

### Generate DDL for creating Explanations Table

In [46]:
# Schema Name where Explanations Table should be created.
EXPLANATIONS_SCHEMA_NAME = None

if ENABLE_EXPLAINABILITY:
    table_ddl_module.generate_explanations_table_ddl(schema_name=EXPLANATIONS_SCHEMA_NAME, table_suffix=NOTEBOOK_RUN_ID, partition_column=PARTITION_COLUMN)

**Create Table SQL Query:** Run the following query, to create a new table
- `CREATE TABLE "explain_five0"."explanations_table" ("request_id" VARCHAR(64) NOT NULL, "scoring_id" VARCHAR(64) NOT NULL, "subscription_id" VARCHAR(64), "data_mart_id" VARCHAR(64), "binding_id" VARCHAR(64), "deployment_id" VARCHAR(64), "asset_name" VARCHAR(256), "deployment_name" VARCHAR(256), "prediction" VARCHAR(64), "created_by" VARCHAR(256), "created_at" TIMESTAMP, "finished_at" TIMESTAMP, "explanation_type" VARCHAR(64), "object_hash" VARCHAR(256), "explanation" BLOB, "status" VARCHAR(64), "explanation_input" BLOB, "explanation_output" BLOB, "error" BLOB, "probability" DOUBLE, "" BIGINT GENERATED ALWAYS AS IDENTITY);`


- `CREATE INDEX "explanations_table_index" ON "explain_five0"."explanations_table" ("subscription_id", "request_id", "scoring_id", "finished_at" DESC)`

**Alter Table SQL Queries:** Run the following queries, in the given order, to add a partition column in your existing table.
1. `ALTER TABLE "explain_five0"."explanations_table" ADD COLUMN "" BIGINT NOT NULL DEFAULT 0;`
2. `ALTER TABLE "explain_five0"."explanations_table" ALTER COLUMN "" DROP DEFAULT;`
3. `ALTER TABLE "explain_five0"."explanations_table" ALTER COLUMN "" SET GENERATED ALWAYS AS IDENTITY;`
4. `CALL SYSPROC.ADMIN_CMD('REORG TABLE "explain_five0"."explanations_table"');`
5. `UPDATE "explain_five0"."explanations_table" SET "" = DEFAULT;`

### Create Configuration Archive
Collect all the artefacts generated above - configuration json, explain archive - and bundle them into an archive. This archive is used as is by IBM Watson OpenScale UI/SDK to onboard model for monitoring. 
UI/SDK will identify the different artefacts and appropriately upload to respective monitors.

In [ ]:
import tarfile
import json

# update flags in configuration json
config["common_configuration"]["enable_explainability"] = ENABLE_EXPLAINABILITY
config["common_configuration"]["enable_fairness"] = ENABLE_FAIRNESS
config["common_configuration"]["enable_quality"] = ENABLE_QUALITY

# write to local
with open("common_configuration.json", "wb") as f:
    f.write(json.dumps(config).encode('utf-8'))

if ENABLE_FAIRNESS:
    # write fairness_statistics.json to local
    with open("fairness_statistics.json", "wb") as f:
        f.write(json.dumps(config.get("fairness_configuration")).encode('utf-8'))

if ENABLE_EXPLAINABILITY:
    # build and write explain archive to local
    from io import BytesIO
    with BytesIO() as archive:
        with tarfile.open(fileobj=archive, mode="w:gz") as tf:
            for filename, filedata in archive_data.items():
                content = BytesIO(filedata.encode("utf8"))
                tarinfo = tarfile.TarInfo(filename)
                tarinfo.size = len(content.getvalue())
                tf.addfile(
                    tarinfo=tarinfo, fileobj=content)

        with open("explainability.tar.gz", "wb") as f:
            f.write(archive.getvalue())

with tarfile.open("configuration_archive.tar.gz", "w:gz") as f:
    # collect all files from local and write to configuration archive
    f.add("common_configuration.json", arcname="common_configuration.json")
    
    if ENABLE_EXPLAINABILITY:
        f.add("explainability.tar.gz", arcname="explainability.tar.gz")
        
    if ENABLE_FAIRNESS:
        f.add("fairness_statistics.json", arcname="fairness_statistics.json")

In [48]:
# create download link for configuration package
from io import BytesIO
import base64

data = None
with open('configuration_archive.tar.gz', 'rb') as f:
    # read configuration archive from local
    data = f.read()

format_args = {
    "payload": base64.b64encode(data).decode(),
    "title": "Download Configuration Archive",
    "filename": "configuration_archive.tar.gz"
}

from IPython.display import HTML
html = '<a download="{filename}" href="data:text/json;base64,{payload}" target="_blank">{title}</a>'
HTML(html.format(**format_args))

#### Authors
Developed by [Prem Piyush Goyal](mailto:prempiyush@in.ibm.com), [Pratap Kishore Varma V](mailto:pvemulam@in.ibm.com)